Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/training/train-within-notebook/train-within-notebook.png)

# Train and deploy a model
_**Create and deploy a model directly from a notebook**_

---
---

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
    1. Viewing run results
    1. Simple parameter sweep
    1. Viewing experiment results
    1. Select the best model
1. [Deploy](#Deploy)
    1. Register the model
    1. Create a scoring file
    1. Create the environment configuration (yml file for Conda and pip packages)
    1. Deploy the as web service on Azure Kubernetes Service (AKS)
    1. Test the Web Service (run and HTTP mehtods)
    1. Clean up


---

## Introduction
Azure Machine Learning provides capabilities to control all aspects of model training and deployment directly from a notebook using the AML Python SDK.  In this notebook we will
* connect to our AML Workspace
* create an experiment that contains multiple runs with tracked metrics
* choose the best model created across all runs
* deploy that model as a service

In the end we will have a model deployed as a web service which we can call from an HTTP endpoint

---

## Setup
Create an Azure Machine Learning servcie in Azure, and launch the studio. 

Create a Workspace, a Compute Instance (VM) and a new Notebook running on that VM as a compute target.  

This example was forked from https://github.com/Azure/MachineLearningNotebooks, and further developed to present an end-to-end example. 

For this notebook we need the Azure ML SDK and access to our workspace.  The following cell imports the SDK, checks the version, and accesses our already configured AzureML workspace. 

See more detail on [Git Integration](https://docs.microsoft.com/en-us/azure/machine-learning/concept-train-model-git-integration#:~:text=Azure%20Machine%20Learning%20provides%20a%20shared%20file%20system,work%20with%20Git%20via%20the%20Git%20CLI%20experience) if you need to upload this notebook in AML.

In [28]:
import azureml.core
from azureml.core import Experiment, Workspace

# Check core SDK version number
print("This notebook was created using version 1.0.2 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")


ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

This notebook was created using version 1.0.2 of the Azure ML SDK
You are currently using version 1.19.0 of the Azure ML SDK

Workspace name: wstw
Azure region: eastus2
Subscription id: 61489282-e75d-4996-b2ef-3126311e55e6
Resource group: tw


---

## Data
We will use the diabetes dataset for this experiement (https://aka.ms/diabetes-data). 

The dataset consists of 8 baseline variables for n=10000 diabetes patients: Pregnancies, PlasmaGlucose, DiastolicBloodPressure, TricepsThickness, SerumInsulin, BMI, DiabetesPedigree, and Age.

The dataset has one dichotomous outcome variable: Diebetic.

In [29]:
from azureml.core import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# load the diabetes dataset from the same folder where this notebook is located
print("Loading Diabetes Data from the CSV file...")
dataset = pd.read_csv('./diabetes.csv')

# Separate features and labels
X, y = dataset[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, dataset['Diabetic'].values
# Split data into training set and test set (80% Training and 20% Testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

data = {
    "train":{"X": X_train, "y": y_train},        
    "test":{"X": X_test, "y": y_test}
}

print ("Data contains", len(data['train']['X']), "training samples and",len(data['test']['X']), "test samples")


Loading Diabetes Data from the CSV file...
Data contains 8000 training samples and 2000 test samples


---
## Train

Let's use scikit-learn to train a simple Ridge regression model.  We use AML to record interesting information about the model in an Experiment.  An Experiment contains a series of trials called Runs.  During this trial we use AML in the following way:
* We access an experiment from our AML workspace by name, which will be created if it doesn't exist
* We use `start_logging` to create a new run in this experiment
* We use `run.log()` to record a parameter, alpha, and an accuracy measure - the Mean Squared Error (MSE) to the run.  We will be able to review and compare these measures in the Azure Portal at a later time.
* We store the resulting model in the **working** directory, which is automatically captured by AML when the run is complete.
* We use `run.complete()` to indicate that the run is over and results can be captured and finalized

In [30]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import joblib

# Get an experiment object from Azure Machine Learning
experiment = Experiment(workspace=ws, name="train-diabetes-regression")

# Create a run object in the experiment
run =  experiment.start_logging()
# Log the algorithm parameter alpha to the run; where alpha is between 0 and 1
run.log('alpha', 0.03)

# Create, fit, and test the scikit-learn Ridge regression model
regression_model = Ridge(alpha=0.03)
regression_model.fit(data['train']['X'], data['train']['y'])
preds = regression_model.predict(data['test']['X'])

# Output the Mean Squared Error to the notebook and to the run
print('Mean Squared Error is', mean_squared_error(data['test']['y'], preds))
run.log('mse', mean_squared_error(data['test']['y'], preds))

# Save the model to the working directory 
model_file_name = 'diabetesregressionmodel.pkl'

joblib.dump(value = regression_model, filename = model_file_name)

# upload the model file explicitly into artifacts 
run.upload_file(name = model_file_name, path_or_stream = model_file_name)

# Complete the run
run.complete()

Mean Squared Error is 0.14922484786002663


### Viewing run results
Azure Machine Learning stores all the details about the run in the Azure cloud.  Let's access those details by retrieving a link to the run using the default run output.  Clicking on the resulting link will take you to an interactive page presenting all run information.

In [31]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
train-diabetes-regression,0d6aa29a-702f-49ca-a6b2-882208bddb17,,Running,Link to Azure Machine Learning studio,Link to Documentation


### Simple parameter sweep
Now let's take the same concept from above and modify the **alpha** parameter.  For each value of alpha we will create a run that will store metrics and the resulting model.  In the end we can use the captured run history to determine which model was the best for us to deploy. 

Note that by using `with experiment.start_logging() as run` AML will automatically call `run.complete()` at the end of each loop.

This example also uses the **tqdm** library to provide a thermometer feedback

In [32]:
import numpy as np
from tqdm import tqdm

# list of numbers from 0 to 1.0 with a 0.10 interval
alphas = np.arange(0.0, 1.0, 0.10)

# try a bunch of alpha values in a Linear Regression (Ridge) model
for alpha in tqdm(alphas):
    # create a bunch of runs, each train a model with a different alpha value
    with experiment.start_logging() as run:
        # Use Ridge algorithm to build a regression model
        regression_model = Ridge(alpha=alpha)
        regression_model.fit(X=data["train"]["X"], y=data["train"]["y"])
        preds = regression_model.predict(X=data["test"]["X"])
        mse = mean_squared_error(y_true=data["test"]["y"], y_pred=preds)

        # log alpha, mean_squared_error and feature names in run history
        run.log(name="alpha", value=alpha)
        run.log(name="mse", value=mse)

        # Save the model to the outputs directory for capture
        joblib.dump(value=regression_model, filename=model_file_name)


100%|██████████| 10/10 [01:28<00:00,  8.84s/it]


### Viewing experiment results
Similar to viewing the run, we can also view the entire experiment.  The experiment report view in the Azure portal lets us view all the runs in a table, and also allows us to customize charts.  This way, we can see how the alpha parameter impacts the quality of the model

In [33]:
# now let's take a look at the experiment in Azure portal.
experiment

Name,Workspace,Report Page,Docs Page
train-diabetes-regression,wstw,Link to Azure Machine Learning studio,Link to Documentation


### Select the best model 
Now that we've created many runs with different parameters, we need to determine which model is the best for deployment.  For this, we will iterate over the set of runs.  From each run we will take the *run id* using the `id` property, and examine the metrics by calling `run.get_metrics()`.  

Since each run may be different, we do need to check if the run has the metric that we are looking for, in this case, **mse**.  To find the best run, we create a dictionary mapping the run id's to the metrics.

Finally, we use the `tag` method to mark the best run to make it easier to find later. 

In [34]:
runs = {}
run_metrics = {}

# Create dictionaries containing the runs and the metrics for all runs containing the 'mse' metric
for r in tqdm(experiment.get_runs()):
    metrics = r.get_metrics()
    if 'mse' in metrics.keys():
        runs[r.id] = r
        run_metrics[r.id] = metrics

# Find the run with the best (lowest) mean squared error and display the id and metrics
best_run_id = min(run_metrics, key = lambda k: run_metrics[k]['mse'])
best_run = runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', run_metrics[best_run_id])

# Tag the best run for identification later
best_run.tag("Best Run")

55it [00:16,  3.25it/s]


Best run is: fe34e27d-a10e-42ed-8367-4c246969bcfe
Metrics: {'alpha': 0.9, 'mse': 0.14922463540047534}


---
## Deploy
Now that we have trained a set of models and identified the run containing the best model, we want to deploy the model for real time inference.  The process of deploying a model involves
* registering a model in your workspace
* creating a scoring file containing init and run methods
* creating an environment settings file describing packages necessary for your scoring file
* creating a deployment configuration (for AKS in this example)
* deploying the model and packages as a web service on an AKS cluster

### Register a model
We have already identified which run contains the "best model" by our evaluation criteria.  Each run has a file structure associated with it that contains various files collected during the run.  Since a run can have many outputs we need to tell AML which file from those outputs represents the model that we want to use for our deployment.  We can use the `run.get_file_names()` method to list the files associated with the run, and then use the `run.register_model()` method to place the model in the workspace's model registry.

When using `run.register_model()` we supply a `model_name` that is meaningful for our scenario and the `model_path` of the model relative to the run.  In this case, the model path is what is returned from `run.get_file_names()`

In [35]:
from azureml.core.model import Model
# View the files in the run
for f in best_run.get_file_names():
    print(f)
    
# Register the model with the workspace
model_path = "diabetesregressionmodel.pkl"
model_name = "diabetesregressionmodel.pkl"
model = Model.register(model_path = model_path,
                       model_name = model_name,
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace =ws)

logs/user_log.txt
Registering model diabetesregressionmodel.pkl


Once a model is registered, it is accessible from the list of models on the AML workspace.  If you register models with the same name multiple times, AML keeps a version history of those models for you.  The `Model.list()` lists all models in a workspace, and can be filtered by name, tags, or model properties.   

In [36]:
# Find all models called "diabetesmodel" and display their version numbers
from azureml.core.model import Model
models = Model.list(ws, name=model_name)
for m in models:
    print(m.name, m.version)

diabetesregressionmodel.pkl 6


### Create a scoring file

Since your model file can essentially be anything you want it to be, you need to supply a scoring script that can load your model and then apply the model to new data. This script is your 'scoring file'. This scoring file is a python program containing, at a minimum, two methods init() and run(). The init() method is called once when your deployment is started so you can load your model and any other required objects. This method uses the get_model_path function to locate the registered model inside the docker container. The run() method is called interactively when the web service is called with one or more data samples to predict.

Important: The schema decorators for pandas and numpy are required to implement the automatic swagger schema generation for input and output variables

After a successful run of the this script, the score.py file be created in the working folder


In [37]:
# upgrade azureml sdk and monitoring
!pip install --upgrade azureml-sdk azureml-monitoring


Requirement already up-to-date: azureml-sdk in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.19.0)
Requirement already up-to-date: azureml-monitoring in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (0.1.0a21)


In [38]:
%%writefile score.py
import json
import pickle
import numpy as np
import pandas as pd
import joblib
from azureml.core.model import Model
from azureml.monitoring import ModelDataCollector # for model data collection

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

def init():
    global model, inputs_datacollection, prediction_datacollection
    # collect inputs data send to, and prediction data received from, the model
    inputs_datacollection = ModelDataCollector(model_name, designation="inputs", feature_names=["Pregnancies", "PlasmaGlucose", "DiastolicBloodPressure", "TricepsThickness", "SerumInsulin", "BMI", "DiabetesPedigree", "Age"])
    prediction_datacollection = ModelDataCollector(model_name, designation="prediction", feature_names=["Diebetic"])
    model_path = Model.get_model_path('diabetesregressionmodel.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

input_sample = pd.DataFrame(data=[{
            "Pregnancies": 0,
            "PlasmaGlucose": 171,
            "DiastolicBloodPressure": 80,
            "TricepsThickness": 34,
            "SerumInsulin": 23,
            "BMI": 43.51,
            "DiabetesPedigree": 1.21,
            "Age": 21,
            }])
output_sample = np.array([0])

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))

def run(data):
    try:
        result = model.predict(data)

        # save the inputs and prediction data collection from this run
        inputs_datacollection.collect(data) 
        prediction_datacollection.collect(result) 

        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


### Create the environment settings 

The environment settings will also be exported into a yml file (myenv.yml) to verify the conda and pip packages.
The yml file will be in the working folder for this deployment (but it is not needed - for verification only)

This step will create the python environment with the required conda and pip packages/dependencies. And then, it will create the inference configuration that will build the Docker container based on the scoring file and the environment configuration. The Docker image is transparent and will be created and registered behind the scenes with the AzureML SDK.

In [39]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment('deploytocloudenv')
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['numpy','scikit-learn'],pip_packages=['azureml-defaults','azureml-monitoring','inference-schema[numpy-support]'])
inference_config = InferenceConfig(entry_script="score.py", environment=env)

with open ("myenv.yml","w") as f:
   f.write(env.python.conda_dependencies.serialize_to_string())


Verify the myenv.yml file in the working folder to ensure it contains the exact following configurations



In [ ]:
# DO NOT RUN THIS STEP - for verification only



# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults~=1.6.0
  - azureml-monitoring
  - inference-schema[numpy-support]
- numpy
- scikit-learn
channels:
- anaconda
- conda-forge

### Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

This step will take a few minutes. If it fails, try again or create the AKS cluster in the studio

In [40]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your AKS cluster
aks_name = 'aks-cluster' 

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


### Deploy a web service with the model to the AKS Cluster


In [41]:
from azureml.core.webservice import AksWebservice

# Set the web service configuration (using default here)
# Data collecoitn is not enabled by default. Enable here
# Application Insight is also not enabled by default. Enable here
aks_config = AksWebservice.deploy_configuration(collect_model_data=True, enable_app_insights=True)

# to disable any deploy configration 
# AksWebservice.update(collect_model_data=False)


In [42]:
%%time
import datetime

aksservice_name ='aks-service'

print(str(datetime.datetime.now()))
# Create the webservice using all of the precreated configurations and our best model
aksservice = Model.deploy(workspace=ws,
                           name=aksservice_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target)

# Wait for the service deployment to complete while displaying log output
aksservice.wait_for_deployment(show_output = True)
print(aksservice.state)
print(aksservice.get_logs)

2020-12-13 19:41:02.117546
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........
Failed


ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 905b98e1-6957-4d32-8aeb-b6bed7fbbad1
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: aks-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image 4fd3004bb6dc44a8ab11621ce638cf35.azurecr.io/azureml/azureml_449f0793b7bb420744de3c145bb8de26:latest locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information."
    },
    {
      "code": "DeploymentFailed",
      "message": "Your container endpoint is not available. P

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 905b98e1-6957-4d32-8aeb-b6bed7fbbad1
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: aks-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image 4fd3004bb6dc44a8ab11621ce638cf35.azurecr.io/azureml/azureml_449f0793b7bb420744de3c145bb8de26:latest locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information."
    },
    {
      "code": "DeploymentFailed",
      "message": 

### Obtain the Swagger URL if successfully deployed

In [43]:
aksservice.swagger_uri

'http://23.101.156.108:80/api/v1/service/aks-service/swagger.json'

### Test web service - run mehtod
Call the web service with some dummy input data to get a prediction.

In [44]:
import json

# Raw dataset (Actual Diabetic is 1)
test_sample = json.dumps({"data": [{
        "Pregnancies": 9,
        "PlasmaGlucose": 103,
        "DiastolicBloodPressure": 78,
        "TricepsThickness": 25,
        "SerumInsulin": 309,
        "BMI": 29.58,
        "DiabetesPedigree": 1.28,
        "Age": 43,}]})

test_sample = bytes(test_sample,encoding = 'utf8')
prediction = aksservice.run(input_data=test_sample)
print(prediction)


# Raw dataset (Actual Diabetic is 0)
test_sample = json.dumps({"data": [{
        "Pregnancies": 0,
        "PlasmaGlucose": 171,
        "DiastolicBloodPressure": 80,
        "TricepsThickness": 34,
        "SerumInsulin": 23,
        "BMI": 43.51,
        "DiabetesPedigree": 1.21,
        "Age": 21,}]})

test_sample = bytes(test_sample,encoding = 'utf8')
prediction = aksservice.run(input_data=test_sample)
print(prediction)


ERROR - Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 503
Headers: {'cache-control': 'max-age=0, private, must-revalidate', 'content-length': '97', 'content-type': 'text/plain; charset=utf-8', 'date': 'Sun, 13 Dec 2020 19:42:33 GMT', 'server': 'Cowboy', 'x-ms-request-id': '0c23724a-df54-41c9-b763-b0c3125740a4'}
Content: b'No ready replicas for service aks-service. \nHelp: https://go.microsoft.com/fwlink/?linkid=2146747'



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 503
Headers: {'cache-control': 'max-age=0, private, must-revalidate', 'content-length': '97', 'content-type': 'text/plain; charset=utf-8', 'date': 'Sun, 13 Dec 2020 19:42:33 GMT', 'server': 'Cowboy', 'x-ms-request-id': '0c23724a-df54-41c9-b763-b0c3125740a4'}
Content: b'No ready replicas for service aks-service. \nHelp: https://go.microsoft.com/fwlink/?linkid=2146747'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 503\nHeaders: {'cache-control': 'max-age=0, private, must-revalidate', 'content-length': '97', 'content-type': 'text/plain; charset=utf-8', 'date': 'Sun, 13 Dec 2020 19:42:33 GMT', 'server': 'Cowboy', 'x-ms-request-id': '0c23724a-df54-41c9

### Test web service - HTTP request
Alternatively you can construct a raw HTTP request and send it to the service. In this case you need to explicitly pass the HTTP header and the API key.


In [ ]:
# if (key) auth is enabled, retrieve the API keys. AML generates two keys.
key1, key2 = aksservice.get_keys()
print(key1)
print(key2)

In [ ]:
# construct raw HTTP request and send to the service
import requests
import json


# Raw dataset (Actual Diabetic is 1)
test_sample = json.dumps({"data": [{
        "Pregnancies": 9,
        "PlasmaGlucose": 103,
        "DiastolicBloodPressure": 78,
        "TricepsThickness": 25,
        "SerumInsulin": 309,
        "BMI": 29.58,
        "DiabetesPedigree": 1.28,
        "Age": 43,}]})
test_sample = bytes(test_sample,encoding = 'utf8')

# If (key) auth is enabled, don't forget to add key to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}
response = requests.post(aksservice.scoring_uri, test_sample, headers=headers)
print("prediction:", response.text)

### Clean up

Delete the ACI instance to stop the compute and any associated billing.

In [ ]:
%%time
aksservice.delete()
model.delete()

<a id='nextsteps'></a>
## Next Steps

In this example, you created a series of models inside the notebook using local data, stored them inside an AML experiment, found the best one and deployed it as a live service!  From here you can continue to use Azure Machine Learning in this regard to run your own experiments and deploy your own models, or you can expand into further capabilities of AML!

If you have a model that is difficult to process locally, either because the data is remote or the model is large, try the [train-on-remote-vm](../train-on-remote-vm) notebook to learn about submitting remote jobs.

If you want to take advantage of multiple cloud machines to perform large parameter sweeps try the [train-hyperparameter-tune-deploy-with-pytorch](../../training-with-deep-learning/train-hyperparameter-tune-deploy-with-pytorch
) sample.

If you want to deploy models to a production cluster try the [production-deploy-to-aks](../../deployment/production-deploy-to-aks
) notebook.